# Webscraping

#### 1- Web Scraping                  :
von Rezensionen von den Seiten Klinikbewertungen.de & Google Maps
#### 2- Data Science/Machine Learning :
Bereinigte Daten mittels einer Machine Learning Methode verarbeiten
#### 3- Web Technologie               :
Webseite/Webapp erstellen, um bezogene Daten und Ergebnisse des Machine Learnings darzustellen

# Klinikbewertungen.de
#### Erst werden Klinikbewertungen.de Daten verarbeitet.
Die benötigte Daten: 
 - Name der Klinik, 
 - Titel, 
 - Datum,
 - Fachbereich, 
 - Erfahrungsbericht,
 - Sternebewertungen :
  - Gesamtzufriedenheit, 
  - Qualität der Beratung, 
  - Mediz. Behandlung, 
  - Verwaltung und Abläufe, 
  - Ausstattung und Gestaltung), 

##### Benötigte Bibliotheken

In [247]:
import pandas as pd
import requests
import scrapy
from scrapy.http import TextResponse
from scrapy.selector import Selector
from bs4 import BeautifulSoup

##### Url Daten von Kliniken in Pandas DataFrame stellen

In [248]:
df = pd.read_excel(r'Klinikliste.xlsx')[:17][["Klinikname","Link Klinikbewertungen"]]
df

,Klinikname,Link Klinikbewertungen
0,Herzogin Elisabeth Hospital,https://www.klinikbewertungen.de/klinik-forum/...
1,Klinik am Zuckerberg,https://www.klinikbewertungen.de/klinik-forum/...
2,Klinikum Wolfsburg,https://www.klinikbewertungen.de/klinik-forum/...
3,Klinikum Peine,https://www.klinikbewertungen.de/klinik-forum/...
4,St. Martini Krankenhaus,https://www.klinikbewertungen.de/klinik-forum/...
5,DIAKOVERE Henriettenstift,https://www.klinikbewertungen.de/klinik-forum/...
6,Eilenriede Klinik Hannover,https://www.klinikbewertungen.de/klinik-forum/...
7,Sophienklinik,https://www.klinikbewertungen.de/klinik-forum/...
8,KRH Klinikum Agness Karll Laatzen,https://www.klinikbewertungen.de/klinik-forum/...
9,Klinikum Wahrendorff,https://www.klinikbewertungen.de/klinik-forum/...


## Wir definieren zu erfassene Daten und erstellen Dataframe

#### Jetzt Klinikbewertungen.de Daten ziehen und dataframe daten ausfüllen

In [249]:
def KlinikData(dataframe, df, zahl): # nimmt leer dataframe, url liste und zahl (max 17) der Kliniken als Parameter
    for klinik in range (zahl):
    
        fach,ausstattung_gestaltung,titel, datum, erfahrung, gesamt, beratung, med_behandlung, verwaltung =[],[],[],[],[],[],[],[],[]

        url = df.iloc[klinik,1]+"/bewertungen?allbew#more" # alle kommentare zu sehen
        print(url)
        r = requests.get(url) 
        response = TextResponse(r.url,body=r.text,encoding="utf-8")

        titel.append(response.xpath("//div[@class='list ratinglist']/article/header/h2/text()").extract())

        datum.append(response.xpath("//div[@class='meta']/time/text()").extract())

        reviews = response.xpath("//div[@class='list ratinglist']/article")            
        fach=[review.xpath(".//span[@class='right']/a/text()").extract() if review.xpath(".//span[@class='right']/a/text()").extract() else review.xpath(".//span[@class='right']/text()").extract() for review in reviews]

        erfahrung.append(response.xpath("//p[@itemprop='reviewBody']").extract()) 
        erfahrung=[BeautifulSoup(i).get_text() for i in erfahrung[0]]

        gesamt.append(response.xpath("//div[@class='list ratinglist']/article/section[@class='rating']/dl/dd[1]/text()").extract())
        beratung.append(response.xpath("//div[@class='list ratinglist']/article/section[@class='rating']/dl/dd[2]/text()").extract())
        med_behandlung.append(response.xpath("//div[@class='list ratinglist']/article/section[@class='rating']/dl/dd[3]/text()").extract())
        verwaltung.append(response.xpath("//div[@class='list ratinglist']/article/section[@class='rating']/dl/dd[4]/text()").extract())

        ratings = response.xpath("//div[@class='list ratinglist']/article")
        ausstattung_gestaltung=[rating.xpath(".//section[@class='rating']/dl/dd[5]/text()").extract() if rating.xpath("..//section/dl/dd[5]/img").extract() else "Null..." for rating in ratings]

        print(df.iloc[klinik,0])

        bewertung_zahl = sum([len(klinik_titel_list) for klinik_titel_list in titel])
        print(bewertung_zahl, 'Bewertungen', end="\n\n")
        for i in range(bewertung_zahl):
            dataframe = dataframe.append({'Name der Klinik':df.iloc[klinik,0],'Fachbereich':fach[i],"Ausstattung und Gestaltung":ausstattung_gestaltung[i], 'Titel': titel[0][i],'Datum':datum[0][i], 'Erfahrungsbericht':erfahrung[i], 'Gesamtzufriedenheit':gesamt[0][i], 'Qualität der Beratung':beratung[0][i],'Mediz. Behandlung':med_behandlung[0][i],'Verwaltung und Abläufe':verwaltung[0][i]}, ignore_index=True)
    print("Total", dataframe.shape[0], "Bewertungen")
    print("Fertig!")
    return(dataframe)
    
    

In [250]:
columns=['Name der Klinik','Titel','Datum', 'Fachbereich','Erfahrungsbericht','Gesamtzufriedenheit','Qualität der Beratung','Mediz. Behandlung','Verwaltung und Abläufe','Ausstattung und Gestaltung']
dataframe=pd.DataFrame(columns=columns)
dataframe= KlinikData(dataframe, df, 17)

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-kliniken-herzogin-elisabeth-braunschweig/bewertungen?allbew#more
Herzogin Elisabeth Hospital
131 Bewertungen

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-klinik-am-zuckerberg-braunschweig/bewertungen?allbew#more
Klinik am Zuckerberg
78 Bewertungen

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-stadtkrankenhaus-wolfsburg/bewertungen?allbew#more
Klinikum Wolfsburg
234 Bewertungen

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-krankenhaus-peine/bewertungen?allbew#more
Klinikum Peine
147 Bewertungen

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-krankenhaus-st-martini-duderstadt/bewertungen?allbew#more
St. Martini Krankenhaus
64 Bewertungen

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-krankenhaus-henriettenstiftung-hannover/bewertungen?allbew#more
DIAKOVERE Henriettenstift
245 Bewertungen

https://www.klinikbewertungen.de/klinik-forum/erfahrung-mit-eilenried

In [251]:
dataframe.shape

(2090, 10)

In [252]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Name der Klinik             2090 non-null   object
 1   Titel                       2090 non-null   object
 2   Datum                       2090 non-null   object
 3   Fachbereich                 2090 non-null   object
 4   Erfahrungsbericht           2090 non-null   object
 5   Gesamtzufriedenheit         2090 non-null   object
 6   Qualität der Beratung       2090 non-null   object
 7   Mediz. Behandlung           2090 non-null   object
 8   Verwaltung und Abläufe      2090 non-null   object
 9   Ausstattung und Gestaltung  2090 non-null   object
dtypes: object(10)
memory usage: 163.4+ KB


#### Dataframe Prüfung

-Die Daten müssen gereinigt werden-

In [253]:
dataframe.head()

,Name der Klinik,Titel,Datum,Fachbereich,Erfahrungsbericht,Gesamtzufriedenheit,Qualität der Beratung,Mediz. Behandlung,Verwaltung und Abläufe,Ausstattung und Gestaltung
0,Herzogin Elisabeth Hospital,Superklinik,07.06.2021,[OrthopÃ¤die],Bestens durchorganisiert.gutes Personal geht i...,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,[sehr zufrieden\n]
1,Herzogin Elisabeth Hospital,Kranken- oder Hotelzimmer?,25.05.2021,[\nUnfallchirurgie\n],Bin nach einem Autounfall im HEH gewesen und h...,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,[zufrieden\n]
2,Herzogin Elisabeth Hospital,Knie OP,11.05.2021,[OrthopÃ¤die],Wurde bereits im Sept. 2020 am Knie operiert. ...,sehr zufrieden (Keine negativen Merkmale im Ge...,sehr zufrieden\n,sehr zufrieden (Sehr gute medizinische und pfl...,sehr zufrieden\n,[sehr zufrieden\n]
3,Herzogin Elisabeth Hospital,Beste Erfahrungen im HEH,09.05.2021,[Chirurgie],Da ich zeit meines Lebens noch nie einen Krank...,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,zufrieden\n,[sehr zufrieden\n]
4,Herzogin Elisabeth Hospital,Superkompetent und freundlich,07.05.2021,[\nGefÃ¤Ãchirurgie\n],Das gesamte Personal von der Notaufnahme Ã¼ber...,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,sehr zufrieden\n,[zufrieden\n]


## Data Cleaning

In [254]:
dataframe['Fachbereich'] = dataframe['Fachbereich'].str[0]
dataframe['Ausstattung und Gestaltung'] = dataframe['Ausstattung und Gestaltung'].str[0]

In [255]:
def reinigung_umlaut(text):
    text= text.str.replace('\n','')
    text= text.str.replace('Ã¶','ö')
    text= text.str.replace('Ã\x84','Ä')
    text= text.str.replace('Ã¤','ä')
    text= text.str.replace('Ã¼','ü')
    text= text.str.replace('Ã\x9f','ß')
    text= text.str.replace('Ã\x9c','Ü')
    return text

dataframe["Name der Klinik"] = reinigung_umlaut(dataframe["Name der Klinik"])
dataframe['Titel'] = reinigung_umlaut(dataframe['Titel'])
dataframe["Fachbereich"] = reinigung_umlaut(dataframe["Fachbereich"])
dataframe["Erfahrungsbericht"] = reinigung_umlaut(dataframe["Erfahrungsbericht"])

In [256]:
dataframe['Fachbereich'] = dataframe['Fachbereich'].str.strip()

dataframe['Gesamtzufriedenheit']=dataframe['Gesamtzufriedenheit'].str.replace(r"\(.*\)","")
dataframe['Gesamtzufriedenheit'] = dataframe['Gesamtzufriedenheit'].str.strip()

dataframe['Qualität der Beratung']=dataframe['Qualität der Beratung'].str.replace(r"\(.*\)","")
dataframe['Qualität der Beratung'] = dataframe['Qualität der Beratung'].str.strip()

dataframe['Mediz. Behandlung']=dataframe['Mediz. Behandlung'].str.replace(r"\(.*\)","")
dataframe['Mediz. Behandlung'] = dataframe['Mediz. Behandlung'].str.strip()

dataframe['Verwaltung und Abläufe']=dataframe['Verwaltung und Abläufe'].str.replace(r"\(.*\)","")
dataframe['Verwaltung und Abläufe'] = dataframe['Verwaltung und Abläufe'].str.strip()

dataframe['Ausstattung und Gestaltung']=dataframe['Ausstattung und Gestaltung'].str.replace(r"\(.*\)","")
dataframe['Ausstattung und Gestaltung'] = dataframe['Ausstattung und Gestaltung'].str.strip()

In [257]:
dataframe

,Name der Klinik,Titel,Datum,Fachbereich,Erfahrungsbericht,Gesamtzufriedenheit,Qualität der Beratung,Mediz. Behandlung,Verwaltung und Abläufe,Ausstattung und Gestaltung
0,Herzogin Elisabeth Hospital,Superklinik,07.06.2021,Orthopädie,Bestens durchorganisiert.gutes Personal geht i...,sehr zufrieden,sehr zufrieden,sehr zufrieden,sehr zufrieden,sehr zufrieden
1,Herzogin Elisabeth Hospital,Kranken- oder Hotelzimmer?,25.05.2021,Unfallchirurgie,Bin nach einem Autounfall im HEH gewesen und h...,sehr zufrieden,sehr zufrieden,sehr zufrieden,sehr zufrieden,zufrieden
2,Herzogin Elisabeth Hospital,Knie OP,11.05.2021,Orthopädie,Wurde bereits im Sept. 2020 am Knie operiert. ...,sehr zufrieden,sehr zufrieden,sehr zufrieden,sehr zufrieden,sehr zufrieden
3,Herzogin Elisabeth Hospital,Beste Erfahrungen im HEH,09.05.2021,Chirurgie,Da ich zeit meines Lebens noch nie einen Krank...,sehr zufrieden,sehr zufrieden,sehr zufrieden,zufrieden,sehr zufrieden
4,Herzogin Elisabeth Hospital,Superkompetent und freundlich,07.05.2021,Gefäßchirurgie,Das gesamte Personal von der Notaufnahme über ...,sehr zufrieden,sehr zufrieden,sehr zufrieden,sehr zufrieden,zufrieden
...,...,...,...,...,...,...,...,...,...,...
2085,Psychiatrische Klinik Lüneburg,Mir gings schlechter als vorher,19.03.2013,Psychiatrie,Ich war vor kuzen Patientrin auf der Station 1...,unzufrieden,unzufrieden,unzufrieden,unzufrieden,NaN
2086,Psychiatrische Klinik Lüneburg,Eine positive Überraschung!,05.12.2009,Psychiatrie,Ich befinde mich seit einigen Wochen in Behand...,sehr zufrieden,sehr zufrieden,zufrieden,sehr zufrieden,NaN
2087,Psychiatrische Klinik Lüneburg,Tagesklinik Uelzener Straße,25.10.2009,Psychiatrie,Ich kann mich Angie nur anschließen! Diese Tag...,unzufrieden,unzufrieden,unzufrieden,unzufrieden,NaN
2088,Psychiatrische Klinik Lüneburg,Tagesklinik I - NO GO!,03.03.2009,Psychiatrie,,unzufrieden,unzufrieden,unzufrieden,weniger zufrieden,NaN


#### Datenspeichern im CSV/JSON-Datenformat im Rechner.

In [258]:
dataframe.to_csv(r'klinik_bewertungen.csv', index = False)

In [260]:
#dataframe.reset_index().to_json(r'klinik_bewertungen.json', orient='records')

# Google Maps
#### Jetzt werden wir  Klinikbewertungen.de Daten verarbeitet.
Die benötigte Daten: 
 - Name der Klinik, 
 - Textuelle Bewertung, 
 - Sternebewertung,
 - Fachbereich, 
 - Datum der Bewertung,
 - Likes

##### Importieren der erforderlichen Bibliotheken

In [263]:
import pandas as pd
from time import sleep
import csv
import bs4 as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import presence_of_element_located


#### Wir definieren zu erfassene Daten unter clmgoogle Variable und erstellen Dataframe für Google Maps Daten

In [264]:
columns_google=['Name der Klinik','Textuelle Bewertung','Sternebewertung', 'Datum der Bewertung','Likes']
dfgoogle=pd.DataFrame(columns=columns_google)
dfgoogle

,Name der Klinik,Textuelle Bewertung,Sternebewertung,Datum der Bewertung,Likes


#### Mit Selenium und Beatiful Soup Klinikdaten scrapen.

In [269]:
df = pd.read_excel(r'Klinikliste.xlsx')[:17][["Klinikname","Link Google Maps"]]
df

,Klinikname,Link Google Maps
0,Herzogin Elisabeth Hospital,https://www.google.com/maps/place/Herzogin+Eli...
1,Klinik am Zuckerberg,https://www.google.com/maps/place/Klinik+Am+Zu...
2,Klinikum Wolfsburg,https://www.google.com/maps/place/Klinikum+Wol...
3,Klinikum Peine,https://www.google.com/maps/place/Klinikum+Pei...
4,St. Martini Krankenhaus,https://www.google.com/maps/place/St.+Martini+...
5,DIAKOVERE Henriettenstift,https://www.google.com/maps/place/DIAKOVERE+He...
6,Eilenriede Klinik Hannover,https://www.google.com/maps/place/Eilenriede+K...
7,Sophienklinik,https://www.google.com/maps/place/Sophienklini...
8,KRH Klinikum Agness Karll Laatzen,https://www.google.com/maps/place/KRH+Klinikum...
9,Klinikum Wahrendorff,https://www.google.com/maps/place/Klinikum+Wah...


In [293]:
google_list = []
def ScrapeGoogle(url):    

    #chrome_driver_path = "chromedriver"
    chrome_options = Options()
    chrome_options.add_argument("--lang=de")
    Options.headless = False 

    driver = webdriver.Chrome(executable_path=r'/Users/fahricigdem/Documents/Refugeeks/Abschlussprojekt/Data Scraping/chromedriver', options=chrome_options)
    
    driver.get(url)
    sleep(5)
    driver.maximize_window()
    
    #cookies zustimmen
    wait = WebDriverWait(driver,5)
    driver.find_element_by_xpath('/html/body/div/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div[1]/div/button/span').click()
    sleep(5)

    klinik_name = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
    print(klinik_name)

    all_review = driver.find_element_by_class_name('widget-pane-link')
    all_review.click()

    sleep(6)
    reviews_number = driver.find_element_by_xpath('//*[@class="gm2-caption"]').text.split()
    scroll_number = (int(reviews_number[0]))//10+1
    print(f"Maximum scroll need to get full data: {scroll_number}")

    #scrollen
    sleep(10)
    print("in scroll after sleep 4")
    x=0
    while (x < int(scroll_number)):
        scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc') # It gets the section of the scroll bar.            
        print( '  ', x+1, '  times scroll')
        sleep(2)
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div) # Scroll it to the bottom.
        x=x+1

    else:
        print('')
    
    #click mehr buttons
    sleep(4)
    mehr_buttons = driver.find_elements_by_css_selector('.ODSEW-KoToPc-ShBeI.gXqMYb-hSRGPd')
    for button in mehr_buttons:
        button.click()
   
    commentators = driver.find_elements_by_xpath('//div[@class="ODSEW-ShBeI-title"]/span')   
    print("commentators ok")
    reviews = driver.find_elements_by_xpath('//div[@class="ODSEW-ShBeI-ShBeI-content"]/span[2]')
    print("reviews ok")
    stars = driver.find_elements_by_xpath('//div[@class="ODSEW-ShBeI-jfdpUb"]/span[2]')
    print("stars ok")
    dates = driver.find_elements_by_xpath('//span[@class="ODSEW-ShBeI-RgZmSc-date"]')
    print("dates ok")
    likes = driver.find_elements_by_xpath('//span[@class="ODSEW-ShBeI-hWJfub-yHKmmc-NnAfwf"]')
    print("likes ok")

    rezension=1
    while(rezension<int(reviews_number[0])):
        klinik_data = []
        klinik_data.append(klinik_name)

        first_review = reviews.pop(0)
        review = first_review.text.replace('\n','**')
        klinik_data.append(review)

        first_date = dates.pop(0)
        date = first_date.text
        klinik_data.append(date)

        if (len(likes))==0:
            like='No Like'
        else:
            first_like = likes.pop(0)
            like = first_like.text
        klinik_data.append(like)

        first_star = stars.pop(0)
        star = first_star.get_attribute("aria-label").split()
        klinik_data.append(star[0])

        google_list.append(klinik_data)
        rezension= rezension+1

    driver.close()

#### Wir nutzen vorherige Funktion um die Daten der 12 Krankenhäusern zu bekommen.

In [294]:
for klinik in range(4,5):
    ScrapeGoogle(df.iloc[klinik,1])
    sleep(10)

St. Martini Krankenhaus
Maximum scroll need to get full data: 6
in scroll after sleep 4
   1   times scroll
   2   times scroll
   3   times scroll
   4   times scroll
   5   times scroll
   6   times scroll

commentators ok
reviews ok
stars ok
dates ok
likes ok


In [295]:
len(google_list)

54

In [96]:
#google_list

In [296]:
dfgoogle

,Name der Klinik,Textuelle Bewertung,Sternebewertung,Datum der Bewertung,Likes


In [297]:
for el in range(len(google_list)):
    dfgoogle = dfgoogle.append({'Name der Klinik':google_list[el][0],'Textuelle Bewertung':google_list[el][1],'Datum der Bewertung':google_list[el][2],'Likes':google_list[el][3], 'Sternebewertung':google_list[el][4]}, ignore_index=True)

In [298]:
dfgoogle

,Name der Klinik,Textuelle Bewertung,Sternebewertung,Datum der Bewertung,Likes
0,St. Martini Krankenhaus,Geschlagene 3 Wochen lag meine Mutter(80+) in ...,1,vor 6 Tagen,1
1,St. Martini Krankenhaus,Nur 2 Stunden...aber die hatten es in sich ;-)...,5,vor 2 Monaten,1
2,St. Martini Krankenhaus,"Ich kann nur positives berichten, lag 5 Tage a...",5,vor 2 Monaten,2
3,St. Martini Krankenhaus,Nach einem Bizepsabriss musste dieser wieder a...,5,vor 3 Monaten,2
4,St. Martini Krankenhaus,Aufgrund akuter Schmerzen hatte ich gestern Na...,5,vor 6 Monaten,1
5,St. Martini Krankenhaus,Ein großes Dankeschön allen Mitarbeitern im Be...,5,vor 2 Monaten,2
6,St. Martini Krankenhaus,In der Notaufnahme und auch auf der Station wa...,5,vor 7 Monaten,1
7,St. Martini Krankenhaus,Leider kann man 0 Sterne nicht geben.**Einfach...,3,vor 5 Monaten,1
8,St. Martini Krankenhaus,Sehr gute Schutzmaßnahmen alle freundlich und ...,5,vor 6 Monaten,2
9,St. Martini Krankenhaus,14 Tage Spontanaufenthalt**Ich habe mich von a...,5,vor 2 Jahren,1


In [100]:
dfgoogle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name der Klinik      1989 non-null   object
 1   Textuelle Bewertung  1989 non-null   object
 2   Sternebewertung      1989 non-null   object
 3   Datum der Bewertung  1989 non-null   object
 4   Likes                1989 non-null   object
dtypes: object(5)
memory usage: 77.8+ KB


In [101]:
dfgoogle.describe

<bound method NDFrame.describe of                      Name der Klinik  \
0        Herzogin Elisabeth Hospital   
1        Herzogin Elisabeth Hospital   
2        Herzogin Elisabeth Hospital   
3        Herzogin Elisabeth Hospital   
4        Herzogin Elisabeth Hospital   
...                              ...   
1984  Psychiatrische Klinik Lüneburg   
1985  Psychiatrische Klinik Lüneburg   
1986  Psychiatrische Klinik Lüneburg   
1987  Psychiatrische Klinik Lüneburg   
1988  Psychiatrische Klinik Lüneburg   

                                    Textuelle Bewertung Sternebewertung  \
0     Vor der Corona Pandemie hätte ich jeden zugest...               2   
1     Tolles Krankenhaus,  trotzdem bin ich lieber g...               4   
2     Lungenembolie!**Ein Tag ITS, phantastische Für...               5   
3                      Alles sehr gut. Volle Punktzahl!               5   
4     Ich werde nie wieder in ein anderes Krankenhau...               5   
...                                

In [110]:
dfgoogle.to_csv(r'klinik_bewertungen_google.csv', index = False)

In [111]:
dfgoogle.reset_index().to_json(r'klinik_bewertungen_google.json', orient='records')